In [ ]:
# 📌 Step 1: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
# 📌 Step 2: Copy zip from Drive to Colab content
!cp "/content/drive/MyDrive/Research/Dataset/Split/Apple/Apple_Split.zip" "/content/Apple_Split.zip"


In [14]:
# 📌 Step 3: Extract ZIP in content
import zipfile
zip_path = "/content/Apple_Split.zip"
extract_dir = "/content/Apple_Split"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)


In [ ]:
!pip install albumentations --quiet


In [15]:
import os
import cv2
import albumentations as A
from tqdm import tqdm
import random

# ✅ Base directory of your extracted split dataset
base_train_dir = '/content/Apple_Split/train'

# ✅ Desired target per class
TARGET_PER_CLASS = 1500

# ✅ Augmentation pipeline
augment = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.3),
    A.Rotate(limit=30, p=0.5),
    A.RandomShadow(p=0.3),
    A.ZoomBlur(p=0.2),
    A.ShiftScaleRotate(p=0.3)
])


/usr/local/lib/python3.11/dist-packages/albumentations/core/validation.py:114: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)


In [16]:
# ✅ Apply augmentation only to classes with fewer than target images
for class_name in os.listdir(base_train_dir):
    class_path = os.path.join(base_train_dir, class_name)
    images = os.listdir(class_path)
    current_count = len(images)

    print(f"📁 {class_name}: {current_count} images")

    if current_count >= TARGET_PER_CLASS:
        continue  # Skip if already enough

    to_generate = TARGET_PER_CLASS - current_count
    img_paths = [os.path.join(class_path, img) for img in images]

    for i in tqdm(range(to_generate), desc=f"🔄 Augmenting {class_name}"):
        img_path = random.choice(img_paths)
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        augmented = augment(image=img)['image']
        save_path = os.path.join(class_path, f'aug_{i}_{os.path.basename(img_path)}')
        cv2.imwrite(save_path, cv2.cvtColor(augmented, cv2.COLOR_RGB2BGR))


📁 Apple___scab: 855 images


🔄 Augmenting Apple___scab: 100%|██████████| 645/645 [00:01<00:00, 538.17it/s]


📁 Apple___healthy: 1799 images
📁 Apple___brown_spot: 150 images


🔄 Augmenting Apple___brown_spot: 100%|██████████| 1350/1350 [00:02<00:00, 530.60it/s]


📁 Apple___alternaria_leaf_spot: 195 images


...

🔄 Augmenting Apple___black_rot: 100%|██████████| 1065/1065 [00:02<00:00, 507.41it/s]


📁 Apple___rust: 869 images


🔄 Augmenting Apple___rust: 100%|██████████| 631/631 [00:01<00:00, 518.03it/s]


📁 Apple___gray_spot: 276 images


🔄 Augmenting Apple___gray_spot: 100%|██████████| 1224/1224 [00:03<00:00, 398.86it/s]


In [13]:
# !rm -rf /content/Apple_Split

In [18]:
import shutil
# zip
shutil.make_archive('/content/apple_augmented', 'zip', '/content/Apple_Split')



'/content/apple_augmented.zip'

In [19]:
# ✅ গন্তব্য ডিরেক্টরি (Drive-এ যেখানে রাখতে চাও)
drive_folder = '/content/drive/MyDrive/Research/Dataset/Augmented/Apple'

# গন্তব্য ফোল্ডার যদি না থাকে, তৈরি করো
os.makedirs(drive_folder, exist_ok=True)

# ✅ কপি করো
shutil.copy('/content/apple_augmented.zip', os.path.join(drive_folder, 'Apple_Augmented.zip'))
print(f"✅ Uploaded to Google Drive: {drive_folder}")

✅ Uploaded to Google Drive: /content/drive/MyDrive/Research/Dataset/Augmented/Apple
